In [1]:
#in v03, I am going to limit the options to games where both teams have sufficient data. 
#in v04, I am going to experiment with row combinations
#i think row combinations is the wrong approach. Going back to column.

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import time
import pickle

In [2]:
from helper_functions.clean_transitions import clean_transition_column
from helper_functions.simulate_game import simulate_game
from helper_functions.combine_matrices import combine_team_matrix

In [3]:
avg_matrix = pd.read_excel('avg_transition_matrix_with_manual_edits.xlsx', sheet_name = 'Mean Pasted', index_col='Starting_State')

In [4]:
# def combine_matrices(matrix_A, matrix_B, avg, params=[]):
#     pass

In [5]:
#limit games to sufficient data
all_games = os.listdir('raw_game_pulls')

In [6]:
games_meta_df = pd.DataFrame()
for g in all_games:
    game_strings = g.split(' vs ')
    team_A = game_strings[0].split('for ')[-1]
    team_B = game_strings[1].split('.xlsx')[0]
    games_meta_df = pd.concat([games_meta_df, pd.DataFrame({'Game':g,
                                                            'Team A':team_A,
                                                            'Team B':team_B}, index=[0])])

In [7]:
games_meta_df=games_meta_df.reset_index(drop=True)

In [8]:
# Define optimization problem
#need to adjust so that all of the algo's use the same games
def optimization_problem_v2(guesses, games_list, x=10):
    global g, guess, distance_score, ou_score, moneyline_score

    all_results=pd.DataFrame()
    
    for g in tqdm(games_list):

        #locate teams to find matrices
        game_res = pd.read_excel('raw_game_pulls/{}'.format(g))
        game_strings = g.split(' vs ')
        team_A = game_strings[0].split('for ')[-1]
        team_B = game_strings[1].split('.xlsx')[0]

        matrix1 = pd.read_excel('team_specific_matrix/{}_A.xlsx'.format(team_A), index_col='Starting_State')
        matrix2 = pd.read_excel('team_specific_matrix/{}_B.xlsx'.format(team_B), index_col='Starting_State')


        # Reshape the parameters into matrices
        for guess in range(len(guesses)):
            m1 = guesses[guess][:18]
            m2 = guesses[guess][18:36]
            m3 = guesses[guess][36:]

            # Construct combined matrix
            # combined_matrix = m1 * matrix1 + m2 * matrix2 + m3 * avg_matrix  # Adjust with scalar multiples
            combined_matrix = matrix1.mul(m1, axis=1) + matrix2.mul(m2,axis=1)+avg_matrix.mul(m3,axis=1)
            
            for r in combined_matrix.index:
                if combined_matrix.loc[r].sum()==0.:
                    combined_matrix.loc[r]=avg_matrix.loc[r].copy()
                
            combined_matrix = combined_matrix.div(combined_matrix.sum(axis=1), axis=0)
            combined_matrix.columns=avg_matrix.columns

            # Evaluate performance on X game simulations
            result = simulate_game(combined_matrix, x)



            distance_score = accuracy(result, game_res)
            ou_score = accuracy_ou(result, game_res)
            moneyline_score = accuracy_winner(result, game_res)

            all_results=pd.concat([all_results, pd.DataFrame({'Game':g,
                                                              'Guess':guess+1,
                                                              'Distance':distance_score,
                                                              'OU':ou_score,
                                                              'Moneyline':moneyline_score}, index=[0])])
    return all_results

In [24]:
g

'6198930_Play-By-Play for Hampton vs Campbell.xlsx'

In [23]:
for g in tqdm(os.listdir('raw_game_pulls')[2731:]):

        game_id = g.split('_Play')[0]
        #locate teams to find matrices
        game_res = pd.read_excel('raw_game_pulls/{}'.format(g))
        game_strings = g.split(' vs ')
        team_A = game_strings[0].split('for ')[-1]
        team_B = game_strings[1].split('.xlsx')[0]

        matrix1 = pd.read_excel('team_specific_matrix/{}_A.xlsx'.format(team_A), index_col='Starting_State')
        matrix2 = pd.read_excel('team_specific_matrix/{}_B.xlsx'.format(team_B), index_col='Starting_State')

        combined_matrix = combine_team_matrix(matrix1,matrix2)

        sim_results = simulate_game(combined_matrix, 1000)

        with open('saved simulations/{}.pickle'.format(game_id), 'wb') as handle:
                pickle.dump(sim_results, handle)


  0%|          | 0/3537 [00:00<?, ?it/s]

 22%|██▏       | 771/3537 [13:15:17<47:33:07, 61.89s/it]


KeyboardInterrupt: 